In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [13]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import requests
driver = webdriver.Chrome()
driver.get('https://www.redbus.in/online-booking/rtc-directory')
time.sleep(3)
driver.maximize_window()
redbus_dic= driver.find_element(By.CSS_SELECTOR,'a[href="/online-booking/hrtc"]')
redbus_dic .click()
HRTC_route_names=[]
HRTC_route_links=[] 
def scrape_data(): 
    route_names= driver.find_elements (By.CLASS_NAME, "route") 
    for route_name in route_names: 
        HRTC_route_names.append(route_name.text)
    route_links= driver.find_elements (By.CLASS_NAME, "route")
    hrefs =[route_link.get_attribute('href') for route_link in route_links] 
    for href in hrefs:
        HRTC_route_links.append(href)
page_tabs = driver.find_elements(By.CLASS_NAME, "DC_117_pageTabs")

print(len(page_tabs))



for i in range(1, len(page_tabs)+1):
    if i > 0:
        x_path = f'/html/body/div[1]/div/div[4]/div[12]/div[{i}]' 
        element=driver.find_element(By.XPATH, x_path)
        driver.execute_script("arguments[0].click();", element)
        scrape_data()
        
print(HRTC_route_names)
print(HRTC_route_links)

5
['Delhi to Shimla', 'Shimla to Delhi', 'Manali to Chandigarh', 'Chandigarh to Manali', 'Delhi to Manali', 'Hamirpur (Himachal Pradesh) to Chandigarh', 'Delhi to Hamirpur (Himachal Pradesh)', 'Delhi to Chandigarh', 'Manali to Delhi', 'Hamirpur (Himachal Pradesh) to Delhi', 'Chandigarh to Hamirpur (Himachal Pradesh)', 'Shimla to Manali', 'Delhi to Dharamshala (Himachal Pradesh)', 'Shimla to Chandigarh', 'Chandigarh to Dharamshala (Himachal Pradesh)', 'Delhi to Baddi (Himachal Pradesh)', 'Dharamshala (Himachal Pradesh) to Chandigarh', 'Chamba (Himachal Pradesh) to Chandigarh', 'Delhi to Dalhousie', 'Delhi to Chamba (Himachal Pradesh)', 'Dalhousie to Delhi', 'Solan to Delhi', 'Delhi to Palampur', 'Dharamshala (Himachal Pradesh) to Delhi', 'Delhi to Solan', 'Chandigarh to Reckong Peo (Himachal Pradesh)', 'Manali to Shimla', 'Palampur to Delhi', 'Chandigarh to Kullu', 'Kangra to Chandigarh', 'Kullu to Chandigarh', 'Delhi to Kangra', 'Chamba (Himachal Pradesh) to Delhi', 'Palampur to Chandi

In [14]:
bus_names = []
bus_types = []
bus_rating = []
dep_time = []
avl_time = []
duration = []
available_seat = []
price = []
route_names = []
route_links = []

def scroll_down():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

for route_name, route_link in zip(HRTC_route_names, HRTC_route_links):
    driver.get(route_link)
    time.sleep(1)
    for details in range(5):
        try:
            view_more_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'button')))
            view_more_button.click()
        except Exception as e:
            print(f"View more button not found for {route_link}: {e}")
            continue

        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            scroll_down()
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        bus_name_elements = driver.find_elements(By.CSS_SELECTOR, 'div[class="travels lh-24 f-bold d-color"]')
        bus_type_elements = driver.find_elements(By.XPATH, '//div[@class="bus-type f-12 m-top-16 l-color evBus"]')
        rating_elements = driver.find_elements(By.XPATH, '//div[@class="rating-sec lh-24"]')
        depature_elements = driver.find_elements(By.XPATH, '//div[@class="dp-time f-19 d-color f-bold"]')
        arrival_elements = driver.find_elements(By.XPATH, '//div[@class="bp-time f-19 d-color disp-Inline"]')
        duration_elements = driver.find_elements(By.XPATH, '//div[@class="dur l-color lh-24"]')
        seat_elements = driver.find_elements(By.XPATH, '//div[@class="column-eight w-15 fl"]')
        rate_elements = driver.find_elements(By.CSS_SELECTOR, 'div[class="fare d-block"]')

        for bus in bus_name_elements:
            bus_names.append(bus.text)
            route_names.append(route_name)
            route_links.append(route_link)
        for bus_type in bus_type_elements:
            bus_types.append(bus_type.text)
        for rating in rating_elements:
            bus_rating.append(rating.text)
        for dep in depature_elements:
            dep_time.append(dep.text)
        for arr in arrival_elements:
            avl_time.append(arr.text)
        for dur in duration_elements:
            duration.append(dur.text)
        for seat in seat_elements:
            available_seat.append(seat.text)
        for fare in rate_elements:
            price.append(fare.text)



max_length = max(len(bus_names), len(bus_types), len(bus_rating), len(dep_time), len(avl_time), len(duration), len(available_seat), len(price))

bus_names.extend([None] * (max_length - len(bus_names)))
bus_types.extend([None] * (max_length - len(bus_types)))
bus_rating.extend([None] * (max_length - len(bus_rating)))
dep_time.extend([None] * (max_length - len(dep_time)))
avl_time.extend([None] * (max_length - len(avl_time)))
duration.extend([None] * (max_length - len(duration)))
available_seat.extend([None] * (max_length - len(available_seat)))
price.extend([None] * (max_length - len(price)))


data = {
    "Route Name": route_names,
    "Route Link": route_links,
    'Bus Name': bus_names,
    'Bus Type': bus_types,
    'Bus Rating': bus_rating,
    'Departure Time': dep_time,
    'Arrival Time': avl_time,
    'Duration': duration,
    'Available Seats': available_seat,
    'Price': price
}

df = pd.DataFrame(data)
df.to_csv('himachal_bus_details.csv', index=False)
print(df.head())


View more button not found for https://www.redbus.in/bus-tickets/delhi-to-shimla: Message: element click intercepted: Element <div class="button">...</div> is not clickable at point (1442, 15). Other element would receive the click: <div class="modify-sec-onward clearfix modify-sec d-color f-bold">...</div>
  (Session info: chrome=126.0.6478.182)
Stacktrace:
	GetHandleVerifier [0x00007FF749D0EEB2+31554]
	(No symbol) [0x00007FF749C87EE9]
	(No symbol) [0x00007FF749B4872A]
	(No symbol) [0x00007FF749BA012E]
	(No symbol) [0x00007FF749B9DAF2]
	(No symbol) [0x00007FF749B9AF8B]
	(No symbol) [0x00007FF749B9A156]
	(No symbol) [0x00007FF749B8C151]
	(No symbol) [0x00007FF749BBD02A]
	(No symbol) [0x00007FF749B8BA76]
	(No symbol) [0x00007FF749BBD240]
	(No symbol) [0x00007FF749BDC977]
	(No symbol) [0x00007FF749BBCDD3]
	(No symbol) [0x00007FF749B8A33B]
	(No symbol) [0x00007FF749B8AED1]
	GetHandleVerifier [0x00007FF74A018B2D+3217341]
	GetHandleVerifier [0x00007FF74A065AF3+3532675]
	GetHandleVerifier [0